In [3]:
import pandas as pd
import numpy as np

In [28]:
file_name = 'stream160919-Last-Set.csv'
odata = pd.read_csv(file_name)
odata = odata[odata.country_name.notnull()]#removing nulls at country.
odata = odata[odata.cohort_id != 99]# removing fake users.

#--------converting the  created_at into datetime and sort by user_id and created_at
odata['created_at'] = pd.to_datetime(odata['created_at'])
odataSort = odata.sort_values(['user_id', 'created_at'])
#odata.groupby("user_id").apply(lambda  x : x)


In [29]:
file_name2 = 'country-and-continent.csv'
continents = pd.read_csv(file_name2)


In [30]:
continents['country_name'] = [country_name.split(",")[0] for country_name in continents['country_name']] #split the extra information
continents.drop_duplicates(subset="country_name")

,Continent_Name,Continent_Code,country_name,Two_Letter_Country_Code,Three_Letter_Country_Code,Country_Number
0,Asia,AS,Afghanistan,AF,AFG,4.0
1,Europe,EU,Albania,AL,ALB,8.0
2,Antarctica,AN,Antarctica (the territory South of 60 deg S),AQ,ATA,10.0
3,Africa,AF,Algeria,DZ,DZA,12.0
4,Oceania,OC,American Samoa,AS,ASM,16.0
5,Europe,EU,Andorra,AD,AND,20.0
6,Africa,AF,Angola,AO,AGO,24.0
7,North America,NaN,Antigua and Barbuda,AG,ATG,28.0
8,Europe,EU,Azerbaijan,AZ,AZE,31.0
10,South America,SA,Argentina,AR,ARG,32.0


In [31]:

# join the data to the get the user 
marget = pd.merge(odataSort,continents, left_on='country_name', right_on='country_name' ,how = 'left')


In [32]:
marget = marget.drop(['subjects','local_time','Continent_Code','Two_Letter_Country_Code','Three_Letter_Country_Code','Country_Number'], axis=1)

In [33]:
marget[marget['country_name'].isnull()]# cheack for nulls

,id,user_id,project,created_at,country_name,city_name,intervention_id,preconfigured_id,cohort_id,algo_info,Continent_Name


In [34]:
def alculate_session_id(odataSort) :
    cond1 = odataSort.created_at-odataSort.created_at.shift(1) > pd.Timedelta(30, 'm')
    cond2 = odataSort.user_id != odataSort.user_id.shift(1)
    odataSort['SessionID'] = (cond1 | cond2).cumsum() # accumulated sim
    return odataSort

In [35]:
 marget= marget.groupby('user_id').apply(lambda x: alculate_session_id(x))

In [12]:
marget

,id,user_id,project,created_at,country_name,city_name,intervention_id,preconfigured_id,cohort_id,algo_info,Continent_Name,SessionID
0,1,1649536,5733,2019-05-23 19:32:14,Poland,Rumia,None,NaN,0,NaN,Europe,1
1,3,1246201,5733,2019-05-23 19:35:06,Greece,NaN,220753619730732286181297752354994884599,1.0,1,0.400477,Europe,1
2,4,1246201,5733,2019-05-23 19:35:38,Greece,NaN,None,0.0,1,0.467576,Europe,1
3,5,1246201,5733,2019-05-23 19:37:03,Greece,NaN,None,0.0,1,0.190000,Europe,1
4,6,1904710,5733,2019-05-23 19:39:00,United States,Philadelphia,84248398304130673939722041248774187823,2.0,2,0.400477,North America,1
5,7,1246201,5733,2019-05-23 19:39:06,Greece,NaN,None,0.0,1,0.280000,Europe,1
6,8,1904710,5733,2019-05-23 19:39:07,United States,Philadelphia,None,0.0,2,0.462258,North America,1
7,9,1904710,5733,2019-05-23 19:39:18,United States,Philadelphia,None,0.0,2,0.398938,North America,1
8,10,1904710,5733,2019-05-23 19:39:47,United States,Philadelphia,None,0.0,2,0.502224,North America,1
9,11,1904710,5733,2019-05-23 19:39:49,United States,Philadelphia,None,0.0,2,0.731942,North America,1


In [36]:
def midCulculations(x):
    Ntask = x.user_id.size
    dor =  (max(x['created_at'])-min(x['created_at']))/np.timedelta64(1, 's')
    dwell = dor/Ntask
    s1 = pd.Series( Ntask,name = 'Ntask')
    s2 = pd.Series( dor,name = 'Doration')
    s3 = pd.Series( dwell,name = 'dwell')
    return pd.concat([s1, s2, s3], axis=1)

In [37]:
sessionTask = marget.groupby(['user_id','SessionID']).apply(lambda x: midCulculations(x))# calculate Ntask per session.
#sessionTask = sessionTask.to_frame()
#sessionTask.columns = ['Ntask']


In [38]:
min(sessionTask['Doration'])

0.0

In [39]:
def Nsessions(x):
    Nsessions =  max(x['SessionID'])
    Continent =  x.Continent_Name.iloc[0]
    s1 = pd.Series(Nsessions, name = 'Nsessions')
    s2 = pd.Series(Continent, name = 'Continent')
    s3 = pd.Series(x['cohort_id'].iloc[0], name = 'cohort_id')
    return pd.concat([s1,s2,s3], axis=1)

In [40]:
usersTable = marget.groupby('user_id').apply(lambda x: Nsessions(x))


In [41]:
 usersTable['AVG_Task_Session'] = sessionTask.groupby('user_id').apply(lambda x: x['Ntask'].mean()).values

In [42]:
 usersTable['AVG_length_Session'] = sessionTask.groupby('user_id').apply(lambda x: x['Doration'].mean()).values

In [43]:
 usersTable['AVG_dwell_Session'] = sessionTask.groupby('user_id').apply(lambda x: x['dwell'].mean()).values

In [44]:
usersTable

,,Nsessions,Continent,cohort_id,AVG_Task_Session,AVG_length_Session,AVG_dwell_Session
user_id,,,,,,,
27,0,6,Europe,1,20.333333,945.000000,42.638528
34,0,1,North America,5,71.000000,6507.000000,91.647887
35,0,7,North America,4,3.857143,137.285714,24.795238
54,0,1,North America,1,2.000000,13.000000,6.500000
59,0,1,Europe,2,9.000000,1437.000000,159.666667
65,0,1,North America,5,10.000000,868.000000,86.800000
76,0,1,Europe,5,9.000000,217.000000,24.111111
85,0,2,Europe,5,10.000000,347.000000,35.119048
86,0,1,Europe,3,37.000000,2141.000000,57.864865


In [48]:
def ContinentData(x):
    s0 = pd.Series(x.size, name = 'Num Of Users')
    s1 = pd.Series(x['Nsessions'].mean(), name = 'avg sessions per user')
    s2 = pd.Series(x['AVG_Task_Session'].mean(), name = 'avg of avg task per user per session')    
    s3 = pd.Series(x['AVG_length_Session'].mean(), name = 'avg of avg of session length per user')    
    s4 = pd.Series(x['AVG_dwell_Session'].mean(), name = 'avg of avg of dwell time per user')    
    return pd.concat([s0,s1,s2,s3,s4], axis=1)

In [49]:
ContinentData = usersTable.groupby('Continent').apply(lambda x: ContinentData(x))


In [50]:
ContinentData.to_csv('ContinentData.csv')

In [52]:
usersTable.to_csv('usersTable.csv')

In [168]:
ContinentData

,,Num Of Users,avg sessions per user,avg of avg task per user per session,avg of avg of session length per user,avg of avg of dwell time per user
Continent,,,,,,
Africa,0,150,1.960000,185.342857,929.684762,5.729178
Asia,0,1734,2.615917,116.370932,626.687212,5.066662
Europe,0,10026,4.474566,151.079639,653.011071,4.424935
North America,0,14916,3.737329,225.353965,848.428051,4.555490
Oceania,0,1986,4.601208,177.817548,781.257889,4.583069
South America,0,624,4.817308,200.337520,908.691780,5.342741


In [25]:
usersTable["AVG_length_Session"].describe()

count     4906.000000
mean       765.965892
std        903.310403
min          0.000000
25%        163.250000
50%        477.250000
75%       1055.250000
max      12556.000000
Name: AVG_length_Session, dtype: float64